In [92]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)

In [93]:
POSSIBLE_VALUES = [2,3,4,5,6,7,8,9,10,11]

DECK_COLUMNS = [str(i) for i in POSSIBLE_VALUES]
HAND_COLUMNS = ["score", "cards", "aces"]

In [94]:
GAME_COLUMNS = ["chance"] + ["player_"+colname for colname in HAND_COLUMNS] + ["dealer_"+colname for colname in HAND_COLUMNS] + DECK_COLUMNS

In [95]:
def get_deck(number_decks=6):
    """get a deck with the specified number of decks (default: 6)"""
    
    deck = pd.Series([number_decks*4]*len(DECK_COLUMNS) ,index=DECK_COLUMNS)
    deck.loc["10"] *= 4
    return deck

In [96]:
def get_game(deck=None, player_cards=[], dealer_cards=[], update_deck=True):
    """get a game with the specified deck, player cards and dealer cards"""

    if deck is None:
        deck = get_deck()
    
    game = pd.DataFrame(
        [1.0] + [0, 0, 0]*2 + deck.values.flatten().tolist(), 
        index=GAME_COLUMNS
    ).T.astype(int).astype({"chance":float})

    for player_card_value in player_cards:
        game = update_player_with_card(game, player_card_value, probabilistic=False, update_deck=update_deck)
    
    for dealer_card_value in dealer_cards:
        game = update_dealer_with_card(game, dealer_card_value, probabilistic=False, update_deck=update_deck)
    
    return game

In [97]:
def update_player_with_card(game, value, probabilistic=True, update_deck=True):
    """update the game with the player drawing a card with the specified value"""

    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["player_score"] += value
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game["player_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    return next_game

In [98]:
def update_dealer_with_card(game, value, probabilistic=True, update_deck=True):
    """update the game with the dealer drawing a card with the specified value"""

    assert np.all(game["dealer_score"] >= 0) and np.all(game["dealer_score"] < 17), f"there are finished hands, but dealer tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["dealer_score"] += value
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game["dealer_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -1
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1 

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

    return next_game

In [99]:
def player_hits_once(game):
    """get updated game when player hits once, throws error for finished hands"""

    assert np.all(game["dealer_cards"] != 0), f"dealer must have one card showing. hands: {game}"    
    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to hit. hands:\n{game}"
    assert game[DECK_COLUMNS].sum(axis=1).max() == game[DECK_COLUMNS].sum(axis=1).min(), f"there are decks with different number of cards. hands:\n{game}"
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["player_score"] += next_game["drawn"]
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "player_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    # removing drawn column
    next_game = next_game.drop("drawn", axis="columns")
    
    return next_game.groupby(game.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()[GAME_COLUMNS].sort_values("player_score")


In [100]:
def player_hits_once_where_possible(game):
    """get updated game when player hits once, finished hands are kept as-is"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  

    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22)
    if is_playable.sum() == 0:
        print("All possible games are finished")
        return game
    elif is_playable.sum() > 0:
        print(f"Only {is_playable.sum()} games are playable")
    
    final = game[~is_playable].copy()
    playable = game[is_playable].copy()

    next_possible = player_hits_once(playable)
    final = pd.concat((final, next_possible), axis=0, ignore_index=True)
    
    return final.groupby(game.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()[GAME_COLUMNS].sort_values("player_score")

In [101]:
def dealer_draws_once(game):
    """get updated game when dealer draws one card, throws error for finished hands"""

    assert np.all(game["dealer_score"] >= 0) and np.all(game["dealer_score"] < 17), f"there are finished hands, but dealer tried to draw. hands:\n{game}"
    
    final = pd.DataFrame([], columns=game.columns)
    playable = game.copy()

    for value in POSSIBLE_VALUES:
        next_possible = update_dealer_with_card(playable, value)
        
        # adding to all possible outcomes
        final = pd.concat((final, next_possible), axis=0, ignore_index=True)
    
    return final.groupby(game.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()[GAME_COLUMNS].sort_values("dealer_score")

In [102]:
def dealer_plays(game):
    """get updated game when dealer plays until 17 or bust"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    is_playable = (game["dealer_score"] >= 0) & (game["dealer_score"] < 17)
    final = game[~is_playable]
    playable = game[is_playable]

    if len(playable) == 0:
        print("Dealer has finished all hands")
        return game

    while len(playable) > 0:
        next_possible = dealer_draws_once(playable)

        is_playable = (next_possible["dealer_score"] < 17) & (next_possible["dealer_score"] >= 0)
        # finished hands are taken out of the game
        final = pd.concat((final, next_possible[~is_playable]), axis=0, ignore_index=True)
        # playable hands
        playable = next_possible[is_playable]
    
    return final

In [103]:
def get_return_chances_finished(game):
    """get the probabilities of each possible return of the current state for the specified game
    no further moves are made"""

    assert np.all((game["dealer_score"] < 0) | (game["dealer_score"] >= 17)), "dealer has not finished yet"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    # blackjack win
    blackjack_win = (game["player_score"] == 22) & (game["dealer_score"] != 22)
    blackjack_win_chance = game.loc[blackjack_win, "chance"].sum()

    # win
    win = (game["player_score"] > game["dealer_score"]) & (game["player_score"] != 22)
    win_chance = game.loc[win, "chance"].sum()

    # lose
    lose = (game["player_score"] < game["dealer_score"])
    lose_chance = game.loc[lose, "chance"].sum()

    # push
    push = (game["player_score"] == game["dealer_score"])
    push_chance = game.loc[push, "chance"].sum()

    expected_return = blackjack_win_chance*1.5 + win_chance*1 + push_chance*0 + lose_chance*(-1)

    return pd.Series({
        "blackjack_win_chance": blackjack_win_chance,
        "win_chance": win_chance,
        "push_chance": push_chance,
        "lose_chance": lose_chance,
        "expected_return": expected_return
    })

In [104]:
def get_return_chances_standing(game, verbose=False):
    """get the expected return of standing (or doing nothing in case the player is already finished)"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = dealer_plays(game.copy())

    if verbose : print(game)

    return get_return_chances_finished(game)

In [105]:
def get_return_chances_hitting_once(game, verbose=False):
    """get the expected return of hitting exactly once, throws error for finished hands"""
    
    assert np.all(game["player_score"] < 22), "player has a natural"
    assert np.all(game["player_score"] >= 0), "player is already busted"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), "Dealer must have exactly one card showing."

    game = player_hits_once(game.copy())

    if verbose : print(game)

    return get_return_chances_standing(game)

In [106]:
def get_return_chances_hitting_once_where_possible(game, verbose=False):
    """Returns the expected return of hitting exactly once where possible"""

    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"  
    
    game = player_hits_once_where_possible(game.copy())

    if verbose : print(game)

    return get_return_chances_standing(game)

In [122]:
game = get_game(deck, [8,4], [7])
scores_stand = get_return_chances_standing(game, True)
scores_hit = get_return_chances_hitting_once(game, True)
print(pd.DataFrame({"standing":scores_stand, "hitting":scores_hit}))

           chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0    3.106796e-01            12             2            0            17             2            0  24  24  23  24  24  23  23  24  95  24
1    7.766990e-02            12             2            0            18             2            1  24  24  23  24  24  23  23  24  96  23
2    2.420880e-02            12             2            0            -1             3            0  24  24  23  23  24  23  23  24  95  24
3    1.210440e-02            12             2            0            -1             3            0  24  24  23  24  23  23  23  23  96  24
4    2.420880e-02            12             2            0            -1             3            0  24  24  23  24  23  23  23  24  95  24
..            ...           ...           ...          ...           ...           ...          ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
509  7.051991e-10   

In [108]:
deck = get_deck()
deck

2     24
3     24
4     24
5     24
6     24
7     24
8     24
9     24
10    96
11    24
dtype: int64

In [109]:
game = dealer_draws_once(game)
game

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11
0,0.074434,12,2,0,4,2,0,22,24,23,24,24,24,23,24,96,24
1,0.077670,12,2,0,5,2,0,23,23,23,24,24,24,23,24,96,24
2,0.074434,12,2,0,6,2,0,23,24,22,24,24,24,23,24,96,24
3,0.077670,12,2,0,7,2,0,23,24,23,23,24,24,23,24,96,24
4,0.077670,12,2,0,8,2,0,23,24,23,24,23,24,23,24,96,24
5,0.077670,12,2,0,9,2,0,23,24,23,24,24,23,23,24,96,24
6,0.074434,12,2,0,10,2,0,23,24,23,24,24,24,22,24,96,24
7,0.077670,12,2,0,11,2,0,23,24,23,24,24,24,23,23,96,24
8,0.310680,12,2,0,12,2,0,23,24,23,24,24,24,23,24,95,24
9,0.077670,12,2,0,13,2,1,23,24,23,24,24,24,23,24,96,23


In [110]:
game = player_hits_once_where_possible(game)
game = player_hits_once_where_possible(game)
game

AssertionError: dealer must have one card showing. hands:
     chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0  0.074434            12             2            0             4             2            0  22  24  23  24  24  24  23  24  96  24
1  0.077670            12             2            0             5             2            0  23  23  23  24  24  24  23  24  96  24
2  0.074434            12             2            0             6             2            0  23  24  22  24  24  24  23  24  96  24
3  0.077670            12             2            0             7             2            0  23  24  23  23  24  24  23  24  96  24
4  0.077670            12             2            0             8             2            0  23  24  23  24  23  24  23  24  96  24
5  0.077670            12             2            0             9             2            0  23  24  23  24  24  23  23  24  96  24
6  0

In [ ]:
scores = get_return_chances_standing(game, True)
scores

              chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0       1.427808e-04            17             2            0            17             2            1  24  24  24  24  23  24  23  23  96  23
1       5.711232e-04            12             2            0            17             2            0  24  23  24  24  24  23  24  23  95  24
2       1.368316e-04            12             2            0            17             2            0  24  23  24  24  24  24  23  22  96  24
3       1.427808e-04             9             2            0            17             2            0  24  23  24  24  23  24  23  23  96  24
4       5.711232e-04             9             2            0            17             2            0  24  24  23  23  24  23  24  24  95  24
...              ...           ...           ...          ...           ...           ...          ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..

blackjack_win_chance    0.045323
win_chance              0.339357
push_chance             0.048233
lose_chance             0.567088
expected_return        -0.159747
dtype: float64

In [ ]:
scores = get_return_chances_hitting_once_where_possible(game, True)
scores

Only 540 games are playable
        chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11
0     0.001625            -2             3            0             2             1            0  22  24  24  24  24  24  24  24  94  24
280   0.000098            -2             3            0             8             1            0  24  24  24  24  24  24  22  22  96  24
279   0.000376            -2             3            0             8             1            0  24  24  24  24  24  24  21  24  95  24
278   0.000094            -2             3            0             8             1            0  24  24  24  24  24  24  21  23  96  24
277   0.000027            -2             3            0             8             1            0  24  24  24  24  24  24  20  24  96  24
...        ...           ...           ...          ...           ...           ...          ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
2202  0.00367

blackjack_win_chance    0.045323
win_chance              0.264836
push_chance             0.047609
lose_chance             0.642231
expected_return        -0.309411
dtype: float64

In [ ]:
deck = get_deck()
player = get_hand(deck, [10,6])
dealer = get_hand(deck, [7])

r_stand = get_expected_return_standing(deck, player, dealer)
r_hit = get_expected_return_hitting_once(deck, player, dealer)

print("")
print("Expected return of standing is", r_stand)
print("Expected return of hitting once is", r_hit)

NameError: name 'get_hand' is not defined

In [ ]:
#TODO: implement chained card probabilities (player draws before dealer, so dealer's probabilities should be conditional on player's card)
#TODO: implement splitting
#TODO: implement doubling down